# pymysql
- 파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
- Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
- https://github.com/PyMySQL/PyMySQL/
- https://pymysql.readthedocs.io/en/latest/

In [3]:
!ls

01 변수와  데이터타입.ipynb
02_자료구조.ipynb
03_제어문.ipynb
04_함수.ipynb
05_객체지향프로그래밍_ (1).ipynb
07_예외처리_.ipynb
09_.ipynb
09_Iterator와 Decorator_.ipynb
10_파이썬 정규표현식_.ipynb
TODO_IO.txt
Untitled.ipynb
Untitled2.ipynb
___01 변수와 데이터타입_.ipynb
___02_자료구조.ipynb
___03_제어문_.ipynb
___04_함수_.ipynb
___05_객체지향프로그래밍_.ipynb
___07_예외처리_.ipynb
___08_입출력_.ipynb
____06_모듈_패키지_import.ipynb
calc.py.ipynb
hello.py
io_todo
io_todo.zip
member.csv
member_2.csv
member_3.csv
my_package
pymysql을 이용해 mysql연동.ipynb
test.txt
test2.txt
test3.txt


# 설치
- 조건
    - python version 3.6 이상
    - mysql version 5.6 이상
- 설치
    - `pip install PyMySQL`
    - `conda install -y -c conda-forge pymysql`

In [1]:
!pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 KB 1.4 MB/s eta 0:00:00
You should consider upgrading via the '/Users/han-yechan/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host="DBMS 서버 ip", 
                                     port="port번호", 
                                     user="계정명", 
                                     password="비밀번호", 
                                     db="연결할데이터베이스이름", 
                                     charset='utf8')
    ```
    - port 번호 기본값: 3306
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
3. Cusror를 이용해 SQL문 전송
    ```python
        cursor.execute("sql문")
    ```
4. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다. 
    ```python
    cursor.close()
    connection.close()
    ```

# 예제

## 테이블 생성
- 참고) pymysql 을 이용해서는 DML(insert, update, delete, update)문을 실행한다. (DDL은 거의 실행하지 않는다.)

In [19]:
#import
import pymysql

#DB 연결 - connect() => 연결 실패시 Exception 실행
connection = pymysql.connect(host = '127.0.0.1',
                             port = 3306,
                             user = 'hanec',
                             password = '1866',
                             db = 'test_db')
print(type(connection))




<class 'pymysql.connections.Connection'>


In [21]:
#Cursor를 Connection으로 부터 얻어온다. = Cursor: sql문 실행을 관리하는 객체.
cursor = connection.cursor()
print(type(cursor))

<class 'pymysql.cursors.Cursor'>


In [26]:
#sql 문 실행 => 쿼리문 실행 실패시(set 문을 잘못 짠 경우, 제약조건 문제) -> Exception 발생
#sql 문은 string을 정의. execute() 메소드를 이용해서 DB에 전송.
sql = """
create table test_user(
id int auto_increment primary key,
name varchar(30) not null,
email varchar(100) not null unique,
tall decimal(5,2),
birthday date
)
"""
cursor.execute(sql) #실행

0

In [27]:
#연결 닫기 - cursor, connection close
cursor.close()
connection.close()

## DML
### insert

In [50]:
import pymysql

sql = "insert into test_user ( name, email, tall, birthday) values('홍길동','hong@a.com', 182.23,'2000-02-03')"
try:
    #연결
    connection = pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8')
    #cursor 생성
    cursor = connection.cursor()
    #insert문 실행
    cnt = cursor.execute(sql)
    
    connection.commit()
    print(f"{cnt}행이 insert 되었습니다.")
except Exception as e:
    if connection:
        connection.rollback()
    print(e)
finally:
# 연결 닫기
    if cursor: #if cursor!=None:
        cursor.close()
    if connection : # !=None:
        connection.close()

(1062, "Duplicate entry 'hong@a.com' for key 'test_user.email'")


In [49]:
#with 문
#with 연결코드 as 변수
sql = "insert into test_user ( name, email, tall, birthday) values('홍길동','hong@a.com', 182.23,'2000-02-03')"
with pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql)
        print(f'{cnt}행이 insert됨')
        connection.commit()

1행이 insert됨


In [51]:
def insert_user(name,email,tall,birthday):
    sql = "insert into test_user ( name, email, tall, birthday) values('홍길동','hong@a.com', 182.23,'2000-02-03')"
    with pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8') as connection:
        with connection.cursor() as cursor:
            cnt = cursor.execute(sql,('유관순','yoo@a.com',175.3,'2000-01-02'))
            print(f'{cnt}행이 insert됨')
            connection.commit()

### Parameterized Query
- Parameterized Query
    - SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 tuple로 제공한다.
    - query문을 쉽게 작성할 수 있는 장점이 있다.        

In [54]:
# datetime 모듈: 날짜-date, 시간-time, 날짜시간 - datetime 등 날짜와 시간을 다루는 모듈. -> DB의 날짜/시간 타입과 연동.
from datetime import date
sql = "insert into test_user ( name, email, tall, birthday) values(%s,%s,%s,%s)"
with pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(sql,('유관순','yoo1@a.com',175.3,'2000-01-02'))
        cnt2 = cursor.execute(sql,('강감찬','kang@a.com',170.3,date(1995,2,3)))
        print(f'{cnt}행이 insert됨')
        print(f'{cnt2}행이 insert됨')
        connection.commit()

1행이 insert됨
1행이 inser됨


### Parameterized Query를 이용해 여러개 행 insert

#### for문 사용

In [ ]:
names = ['이름1','이름2','이름3']
emails = ['e1@a.com','e2@.com','e3@.com']


#### executemany() 사용
- insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.

In [60]:
datas = [
    ['name1','a1@a.com',172.3,'2000-02-22'],
    ['name2','a2@a.com',182.44,'2001-03-20'],
    ['name3','a3@a.com',162.44,'2005-03-10'],
]
sql = 'insert into test_user(name,email,tall,birthday) values (%s,%s,%s,%s)'
with pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cursor.executemany(sql,datas)
        connection.commit()

IntegrityError: (1062, "Duplicate entry 'a1@a.com' for key 'test_user.email'")

### update/delete
- 코딩 절차는 insert 와 동일

In [61]:
update_sql = "ubdate test_user set tall = tall + 10, birthday = '2000-01-01'"
with pymysql.connect(host='127.0.0.1',port=3306,user='hanec',password='1866',db='test_db',charset='utf8') as connection:
    with connection.cursor() as cursor:
        cnt = cursor.execute(update_sql)
        connection.commit()
        print(f'{cnt}행이 update 됨')

ProgrammingError: (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'ubdate test_user set tall = tall + 10, birthday = '2000-01-01'' at line 1")

## select (DQL)
- 조회결과 조회
    - cusor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
- fetch메소드
    - **fetchall()**
        - 조회한 모든 행을을 반환
    - **fetchmany(size=개수)**
        - 지정한 size개수 만큼 반환
    - **fetchone()**
        - 조회결과 중 첫번째 행만 반환
        - 주로 pk 동등 조건으로 조회한 경우 사용

### fetchall()

### fetchone()

### fetchmany()

### curor 는 iterable 타입
- for문에 select 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.